# **1. Mostrar los diferentes tipos de aeropuertos.**

In [ ]:
import csv
from pathlib import Path

path_airports = Path("../custom_data/ar-airports-custom.csv")

# Manejo de excepciones por si el archivo no es encontrado.
try:   
    with path_airports.open('r') as file_airports:
        reader = csv.reader(file_airports)
        
        # Descarto el header del csv.
        next(reader) 

        # En esta lista guardare los tipos de aeropuertos que ya he mostrado en pantalla.
        listed = []
        for type in reader:

            # Recorro el reader, en caso de que sea un type que no he mostrado se lista en pantalla.
            if ( not type[2] in listed):
                print(type[2])
                listed.append(type[2])
except FileNotFoundError:
    print(f'El archivo en {path_airports} no fue encontrado')

## **2. Mostrar aeropuertos según la elevacion que elija el usuario.**

In [ ]:
import csv
from pathlib import Path

path_airports = Path("../custom_data/ar-airports-custom.csv")

# Manejo de excepciones.
try:
    with (path_airports.open('r', encoding='utf-8') as file_airports):
        reader = csv.reader(file_airports)

        # Descarto el header.
        next(reader)

        # Pido el tipo de elevacion que se quiera enlistar.
        value = input("""Ingrese "bajo","medio",o "alto" según el tipo de elevacion 
                      que quiere tener en cuenta al momento de listar en pantalla los aeropuertos""").lower() 
        
        # Verifico que el usuario haya ingresado una de las opciones correctamente.
        while not (value == 'bajo' or value == 'medio' or value == 'alto'):
            print('Valor invalido.')
            value = input("""Ingrese "bajo","medio",o "alto" según el tipo de elevacion 
                      que quiere tener en cuenta al momento de listar en pantalla los aeropuertos""").lower()
            
        # Comparo la columna de elevation con el value ingresado y en caso de igualar imprimo.
        for rowh in reader:
            if (rowh[23] == value):
                print(rowh[3])
except FileNotFoundError:
    print(f'En la ruta {path_airports} no se encuentra el archivo buscado.')

## **3. Mostrar los aeropuertos con elevacion mayor o menor a un valor dado.**

In [19]:
import csv
from pathlib import Path

path_airports = Path('../custom_data/ar-airports-custom.csv')

# Manejo de excepciones.
try:
    with (path_airports.open('r', encoding='utf-8') as file_airports):
        reader = csv.reader(file_airports)
        next(reader)
        
        # Pido los valores para la comparacion.
        value = float(input('Ingrese la elevacion con la que quiere comparar, utilice metros como la unidad de medida')) * 3.281
        # A VALUE LO MUTIPLICO POR 3.281 PARA CONVERTIRLO A LA UNIDAD DE MEDIDA PIES, LA UTILIZADA EN EL CSV.
        comparated = input('Ingrese "mayor" o "menor" segun los aeropuertos que quiere listar en pantalla').lower()

        for rowh in reader:

            # Verifico que el dato no este vacio.
            if rowh[6] != '':
                ft = float(rowh[6])
                
                # Separo los casos en los que se quiere mostrar el mayor o el menor.
                if comparated == 'mayor':
                    if ft >= value:
                        print(rowh[3], (ft / 3.281))
                elif comparated == 'menor':
                    if ft <= value:
                        print(rowh[3], (ft / 3.281))
except FileNotFoundError:
    print(f'Archivo en {path_airports} no encontrado')

## **4. Mostrar los aeropuertos, lagos y tipo de conectividad en provincias con población mayor o menor a un valor ingresado.**

In [ ]:
import csv 

import sys
from pathlib import Path

sys.path.append(str(Path('..').resolve()))

from data_process import consulta_service

# Se ingresan los datos necesarios, verificando que sean válidos.
while True:
    poblacion_input = input('Ingrese la poblacion que desea: ')
    # Verifico si la entrada es un número
    if poblacion_input.isdigit():
        poblacion = int(poblacion_input)
        # Verificar si la población es un valor válido
        if poblacion >= 0: 
            break
        else:
            print("Error: La población no puede ser un valor negativo.")
    else:
        print("Error: Por favor, ingrese un número entero válido.")

while True:
    signo = input('Ingrese "<" para menor, o ">" para mayor: ')
    if signo in ['<', '>']:
        break
    else:
        print ('Caracter invalido.')
  
path_lagos = Path('../custom_data/lagos_arg.csv')
path_aeropuertos = Path('../custom_data/ar-airports-custom.csv')
path_conectividad = Path('../custom_data/Conectividad_internet_modificado.csv')
path_censo_provincias = Path('../datasets/c2022_tp_c_resumen_adaptado.csv')


try:
    with (path_censo_provincias.open (mode='r', encoding='UTF-8') as file_censo_provincias):
        csv_censo_provincia = csv.reader(file_censo_provincias)
        next(csv_censo_provincia)

        #SACO TOTAL PAIS
        next(csv_censo_provincia)
        

        #OBTENGO NOMBRE DE LAS PROVINCIAS QUE CUMPLEN LA CONDICION 
        provincias_validas = consulta_service.filtrar_provincias(csv_censo_provincia, poblacion, signo)
        
        #SI RIO NEGRO ESTA EN LA LISTA LE AGRREGO TILDE PARA UNIFICAR
        provincias_validas = ['Río Negro' if x == 'Rio Negro' else x for x in provincias_validas]

except FileNotFoundError:
    print("No se pudo encontrar el archivo del censo.")
except IOError:
    print("Error al abrir el archivo del censo.")
    

# OBTENGO LAGOS ------------------------------------
try:
    with (path_lagos.open (mode='r', encoding='UTF-8')) as file_lagos:
        csv_lagos = csv.reader(file_lagos)
        next(csv_lagos)

        print ('--------------------LAGOS----------------------')

        lagos = consulta_service.obtener_lagos (provincias_validas, csv_lagos)
        for lago in lagos:
            print (lago)

except FileNotFoundError:
    print("No se pudo encontrar el archivo de lagos.")
except IOError:
    print("Error al abrir el archivo de lagos.")


# OBTENGO AEROPUERTOS ------------------------------
try:
    with (path_aeropuertos.open (mode='r', encoding='UTF-8') as file_aeropuertos):
        csv_aeropuertos = csv.reader(file_aeropuertos)
        next(csv_aeropuertos)

        print ('--------------------AEROPUERTOS----------------------')
        PROV_NAME = 24
        NAME = 3
        for aeropuerto in csv_aeropuertos:
            if aeropuerto[PROV_NAME] in provincias_validas:
                print (aeropuerto[NAME])

except FileNotFoundError:
    print("No se pudo encontrar el archivo de aeropuertos.")
except IOError:
    print("Error al abrir el archivo de aeropuertos.")


# OBTENGO TIPOS DE CONECTIVIDAD: -------------------
try:
    with (path_conectividad.open (mode='r', encoding='UTF-8') as file_conectividad):
        csv_conectividad = csv.DictReader(file_conectividad)

        print ('--------------------CONECTIVIDAD----------------------')

        #UNIFICO LOS NOMBRES DE LAS PROVINCIAS
        provincias_validas = ['caba' if x == 'Ciudad Autónoma de Buenos Aires' else x for x in provincias_validas]
        provincias_validas = map(consulta_service.sin_tilde, provincias_validas)
        provincias_validas = map(str.upper, provincias_validas)


        provincias_validas_conectividad =list( map (lambda x: {'Provincia': x, 'ADSL':'NO', 'CABLEMODEM': 'NO', 'DIALUP': 'NO', 'FIBRAOPTICA': 'NO', 'SATELITAL': 'NO', 'WIRELESS': 'NO', 'TELEFONIAFIJA': 'NO', '3G': 'NO', '4G': 'NO'}, provincias_validas))

        for localidad in csv_conectividad:
            provincia_a_actualizar = next((prov for prov in provincias_validas_conectividad if prov['Provincia'] == localidad['Provincia']), None)
            if provincia_a_actualizar is not None:
                consulta_service.actualizar_conectividad (localidad, provincia_a_actualizar)


        for provincia in provincias_validas_conectividad:
            print (provincia)

except FileNotFoundError:
    print("No se pudo encontrar el archivo de conectividad.")
except IOError:
    print("Error al abrir el archivo de conectividad.")

## **5. Mostrar los aeropuertos en las capitales de cada provincia.**

In [ ]:
import csv
import sys

from pathlib import Path
sys.path.append(str(Path('..').resolve()))

from data_process import consulta_service

NAME = 3
MUNICIPALITY = 13

path_ar = Path('../datasets/ar.csv')
path_aerppuerto = Path('../custom_data/ar-airports-custom.csv')
try:
    with (
        path_aerppuerto.open(mode='r', encoding='UTF-8') as file_aerpuertos,
        path_ar.open(mode='r', encoding='UTF-8') as file_ar       
        ):

        csv_aerpuertos = csv.reader(file_aerpuertos)
        csv_ar = list(csv.reader(file_ar))

        #SACO EL HEADER
        next(csv_aerpuertos)

        #RECORRO LOS AEROPUERTOS Y CHEQUE SI SE UBICAN EN LA CAPITAL 
        for aeropuerto in csv_aerpuertos:
            if consulta_service.esta_en_la_capital(aeropuerto[MUNICIPALITY], csv_ar) :
                print (f'Ciudad Capital: {aeropuerto[MUNICIPALITY]}, - Aerpuerto: {aeropuerto[NAME]}')
                
except FileNotFoundError:
    print("El archivo CSV no se encontró en la ruta especificada.")
except Exception as e:
    print("Ocurrió un error:", e)

## **6. Mostrar los Lagos de una superficie ingresada por el usuario según la columna 'Sup Tamaño'.**

In [ ]:
import csv
from pathlib import Path

tamaño = input('Elija tamaño (chico, medio o grande): ').lower()

while tamaño not in ['chico', 'medio', 'grande']:
    print("Tamaño inválido. Por favor, elija entre 'chico', 'medio' o 'grande'.")
    tamaño = input('Elija tamaño (chico, medio o grande): ').lower()

NOMBRE = 0 
SUP_TAMAÑO = 9

path_lagos = Path('../custom_data/lagos_arg.csv')

try:
    with (path_lagos.open(mode='r', encoding='UTF-8')) as file_lagos:
        csv_lagos = csv.reader(file_lagos)
        
        #SACO HEADER
        next(csv_lagos)
        
        print (f'Lagos con tamaño {tamaño}: ')
        for lago in csv_lagos:
            if lago[SUP_TAMAÑO] == tamaño:
                print(lago[NOMBRE])

except FileNotFoundError:
    print("El archivo CSV no se encontró en la ruta especificada.")
except Exception as e:
    print("Ocurrió un error:", e)


## **7. Listar las 5 Jurisdicciones con mayor porcentaje de población en situación de calle.**

In [ ]:
import csv
from pathlib import Path

path_csv_censo = Path('../custom_data/c2022_tp_c_resumen_adaptado_custom.csv')

JURISDICCION = 0
TOTAL_POBLACION = 1
POBLACION_EN_SITUACION_DE_CALLE = 4

try:

  with (path_csv_censo.open(mode='r', encoding='UTF-8')) as file_censo:
    csv_censo = csv.reader(file_censo)
    next(csv_censo)
    next(csv_censo)

    # Lista para almacenar los porcentajes de población en situación de calle por jurisdicción
    lista_porcentajes = []

    for provincia in csv_censo:
        # Calculo porcentaje de poblacion en situacion de calle
        porcentaje = (float(provincia[POBLACION_EN_SITUACION_DE_CALLE]) / float(provincia[TOTAL_POBLACION])) * 100
       
        # Ingreso el porcentaje la lista de diccionarios
        porcentaje_jurisiccion = {'jurisdiccion': provincia[JURISDICCION], 'porcentaje': porcentaje}
        lista_porcentajes.append(porcentaje_jurisiccion)

  # Ordenar la lista de porcentajes por el porcentaje de población en situación de calle de mayor a menor y seleccionar los primeros 5
  lista_porcentajes_ordenada = sorted(lista_porcentajes, key=lambda x: x['porcentaje'], reverse=True)[:5]

  for jurisdiccion in lista_porcentajes_ordenada:
    print (jurisdiccion)

except FileNotFoundError:
    print("El archivo CSV no se encontró en la ruta especificada.")
except Exception as e:
    print("Ocurrió un error:", e)

## **8. Mostrar la juridiccion donde la brecha entre hombres y mujeres sea mayor.**

In [ ]:
import csv
from pathlib import Path

path_censo = Path('../data/c2022_tp_c_resumen_adaptado.csv')

juri = 0
total_poblacion = 1
poblacion_hombre = 5
poblacion_mujer = 9

with(path_censo.open('r', encoding='utf-8')) as file_censo:
    reader = csv.reader(file_censo)
    # Me deshago del header y de la primera linea del csv que habla de todo el pais.
    next(reader)
    next(reader)

    # Utilizare variables para guardar el nombre de la juridiccion y el porcentaje diferencial.
    max_diferencia = {
        'juri' : '',
        'porcentaje' : 0.0
    }

    for line in reader:

        # Realizo una expresion lambda para hacer mas legible el codigo.
        calculo = lambda x: (float(x) * 100) / float(line[total_poblacion])
        porcentaje_diferencial= calculo(line[poblacion_hombre]) - calculo(line[poblacion_mujer])

        # En caso de haber conseguido un valor negativo (hecho en caso de que el calculo de las mujeres sea un numero mayor al de hombres)
        # . lo multiplico por -1 para eliminar su signo negativo.
        if porcentaje_diferencial < 0:
            porcentaje_diferencial *= -1
        
        # Comparo el porcentaje calculado con mi procentaje maximo.
        if (porcentaje_diferencial > max_diferencia['porcentaje']):
            max_diferencia['juri'] = line[juri]
            max_diferencia['porcentaje'] = porcentaje_diferencial
        
    print (max_diferencia)

## **9. Mostrar los diferentes tipos de conectividades**

In [ ]:
try:
    path = Path("../custom_data/Conectividad_internet_modificado.csv")

    # Conjunto para almacenar los tipos de conectividad únicos
    tipos_conectividad = set()

    # Leer el archivo CSV y obtener el header
    with path.open(newline='', encoding='UTF-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)

    # Obtener los tipos de conectividad del header
    for columna in header[4:-4]:
        tipos_conectividad.add(columna)

    # Mostrar los tipos de conectividad
    print("Tipos de conectividades:")
    for tipo in tipos_conectividad:
        print(tipo)

except FileNotFoundError:
    print("No se pudo encontrar el archivo.")

## **10. Mostrar las cantidades de localidades con cada tipo de conectividad**

In [ ]:
path = Path("../custom_data/Conectividad_internet_modificado.csv")

conectividad_localidad = {
    "ADSL": 0,
    "CABLEMODEM": 0,
    "DIALUP": 0,
    "FIBRAOPTICA": 0,
    "SATELITAL": 0,
    "WIRELESS": 0,
    "TELEFONIAFIJA": 0,
    "3G": 0,
    "4G": 0
}

try:
    # Leer el archivo CSV y contar las localidades con cada tipo de conectividad
    with path.open(mode='r', newline='', encoding='UTF-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            for conectividad in conectividad_localidad:
                if row[conectividad] == "SI":
                    conectividad_localidad[conectividad] += 1

    # Imprimir las localidades con cada tipo de conectividad
    for conectividad, cantidad in conectividad_localidad.items():
        print(f"Localidades con {conectividad}: {cantidad}")
except FileNotFoundError:
    print("No se pudo encontrar el archivo.")

## **11. Mostrar las provincias para las cuales todas sus ciudades poseen FIBRA ÓPTICA**

In [ ]:
path = Path("../custom_data/Conectividad_internet_modificado.csv")

# Diccionario para almacenar las ciudades por provincia
ciudades_por_provincia = {}

try:
    # Leer el archivo CSV y almacenar las ciudades por provincia
    with path.open(newline='', mode='r', encoding='UTF-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            provincia = row["Provincia"]
            localidad = row["Localidad"]
            fibra_optica = row["FIBRAOPTICA"]
            if provincia not in ciudades_por_provincia:
                ciudades_por_provincia[provincia] = set()
            ciudades_por_provincia[provincia].add((localidad, fibra_optica))

    # Mostrar las provincias donde todas las ciudades poseen FIBRA ÓPTICA
    print("Provincias donde todas las ciudades poseen FIBRA ÓPTICA:")
    for provincia, ciudades in ciudades_por_provincia.items():
        todas_con_fibra_optica = all(fibra_optica == "SI" for _, fibra_optica in ciudades)
        if todas_con_fibra_optica:
            print(provincia)
except FileNotFoundError:
    print("No se pudo encontrar el archivo.")

## **12. Mostrar para cada provincia su capital y, si se conoce la información para dicha capital, informar si posee conectividad**

In [ ]:
path_conectividad = Path("../custom_data/Conectividad_internet_modificado.csv")
path_capitales = Path("../datasets/ar.csv")
# Guardar en un diccionario donde la clave es la localidad y el valor es la conectividad, en caso en caso de ser vacio, imprimir conectividad desconocida
# Diccionario para almacenar las capitales y sus provincias
capitales_provincias = {}

# Leer el archivo de capitales y guardar las capitales en el conjunto y en el diccionario
with path_capitales.open(mode='r', newline='', encoding='UTF-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row["capital"] in ["admin", "primary"]:
            capital = row["city"]
            provincia = row["admin_name"]
            capitales_provincias[capital] = provincia

# Nuevo diccionario para almacenar las claves y valores modificados
nuevo_capitales_provincias = {}

# Recorrer el diccionario original y reemplazar las vocales con tilde por sus versiones sin tilde
for clave, valor in capitales_provincias.items():
    nuevo_valor = valor.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
    nuevo_capitales_provincias[clave] = nuevo_valor

# Reemplazar el diccionario original con el nuevo
capitales_provincias = nuevo_capitales_provincias

# Mostrar para cada provincia su capital
print("Provincias y su respectiva capital")
for clave, valor in capitales_provincias.items():
    print(f"Provincia: {valor}, Capital: {clave}")

# Diccionario vacio donde la clave es la provincia y el valor es posee_conectividad
conectividad_capital = {}

# Copiar las claves del diccionario capitales_provincias a conectividad_provincia con valores nulos
conectividad_capital.update({capital: None for capital in capitales_provincias})

# Verificar la conectividad de las capitales de las provincias
print("Conectividad de las provincias:")
with path_conectividad.open(mode='r', newline='', encoding='UTF-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        provincia = row["Provincia"]
        localidad = row["Localidad"]
        conectividad = row["posee_conectividad"]

        if localidad in capitales_provincias:
            if capitales_provincias[localidad].lower() == provincia.lower():
                conectividad_capital[localidad] = conectividad

# Imprimir la conectividad de las capitales
print("Conectividad de las provincias (Capitales):")
for capital, conectividad in conectividad_capital.items():
    if conectividad is None:
        print(f"Capital: {capital}, Conectividad: Conectividad desconocida")
    else:
        print(f"Capital: {capital}, Conectividad: {conectividad}")